# 案例介绍

目标跟踪是计算机视觉领域的一个重要的研究方向，通过分析图像序列，检测并跟踪识别目标的运动状态，如位置、运动轨迹。目标跟踪为视觉导航、位姿估计与运动分析等方向提供了原始数据。因此在智能监控、机器人导航等领域应用广泛。比如目标跟踪是机器人感知外界环境并作出反应的基础，是理解图像的关键。

本案例介绍了多种单目标跟踪算法的核心思想及实现方式，并比较了其跟踪效果。

In [ ]:
[目标跟踪.png]

# 目录
[1. 单目标跟踪算法介绍](#1)<br>
[2. TLD](#2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 光流介绍](#2.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 稀疏点光流](#2.2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 稠密光流](#2.3)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.4 算法实现](#2.4)<br>
[3. MOSSE](#3)<br>
[4. KCF](#4)<br>
[5. 总结](#5)<br>


<div id="1"></div>
# 1  单目标跟踪算法介绍

根据跟踪目标的数量，可以将目标跟踪算法分为单目标跟踪，以及多目标跟踪。就单目标跟踪而言，算法需要考虑视频序列中，目标的位置、大小、外观的变化等，且跟踪效果会受到光照、遮挡、运动方式等因素影响。

早期的单目标跟踪算法集中于生成式模型跟踪算法的研究，如光流法，Meanshift算法等。算法首先建立目标模型或者提取目标特征，在后续帧中进行搜索与目标相似的物体，实现目标跟踪。

这类方法的缺点在于无法充分利用图像的背景信息，无法适应目标的外观变化。但其相关思想仍十分重要，如光流识别仍是计算机视觉中的重要研究领域。同时，如今提出了效果更良好的跟踪算法：相关滤波算法，如MOSSE、KCF等。


接下来我们先介绍基于光流识别的TLD算法，再将目光放在如今更为主流的目标跟踪算法，如MOSSE等。

<div id="2"></div>
# 2 TLD

TLD算法全称为Tracking-Learning-Detection，是一种单目标长时间跟踪算法。它分为跟踪模块（Tracker）与检测模块（Detector），同时加入了学习模块（Learning）：不断更新跟踪模块的“显著特征点”（角点），以及检测模块的目标模型和参数，使得跟踪的效果更佳稳定、可靠。

跟踪模块：根据目标物体在视频前一帧中的位置，预测其在当前帧中的位置。

检测模块：在当前帧中根据目标物体的先验信息，如物体的形状色彩、各种特征等，检测目标物体在当前帧中的位置。

学习模块：在不断预测的过程中不断更新正、负样本库，用于后续帧中目标物体的位置检测。其中正样本为目标框，负样本为除目标框外的部分图像。

三个模块之间的相互作用过程为：

跟踪模块 -> 学习模块：跟踪模块产生物体运动的轨迹，为学习模块提供正样本。 

检测模块 -> 跟踪模块：检测模块估计跟踪模块的误差，如果误差很大，则改正跟踪模块的结果。

具体过程为：检测模块对每一帧图像进行检测，找到与目标物体相似的所有外观的位置，产生正样本和负样本，交给学习模块。算法从所有正样本中选出一个最可信的目标位置，作为这一帧TLD算法的输出结果。再更新跟踪模中目标物体的起始位置。 

学习模块 -> 检测模块：学习模块根据跟踪模块和检测模块产生的正负样本，迭代训练分类器，改善检测模块的精度。

In [ ]:
[TLD.png]

<div id="2.1"></div>
## 2.1 光流介绍

TLD的跟踪模块基于中值流跟踪算法（Median Flow tracker），该算法基于LK光流跟踪算法。

光流表达了图像的变化，包含了目标运动的信息。当人们观察运动物体时，物体的景象在人眼的视网膜上形成一系列连续变化的图像，这一系列连续变化的信息不断“流过”视网膜（即图像平面），好像一种光的“流”，故称之为光流。

在计算机视觉中，光流描述了视频中连续图像之间的关联关系。在时间间隔很小，比如视频的连续前后两帧之间时，光流等同于描述目标点的位移。这个位移可以由目标点本身的移动、相机的运动，或者两者的共同运动所产生。




<div id="2.2"></div>
## 2.2 稀疏点光流

根据是否选取图像稀疏点，光流估计可以分为稀疏光流和稠密光流。如下图，左图是稀疏光流的示意图：选取特征明显，即梯度较大的点进行光流估计和跟踪。右图为连续帧稠密光流的估计结果，计算了图像上所有像素点的运动。

稀疏光流估计最经典的算法是Lucas Kanade(KL)算法，下列介绍如何通过OpenCV实现稀疏点光流估计。

首先加载需要调用的库，便于后续进行函数调用。

In [2]:
import cv2
import numpy as np

接下来读取需要进行光流估计的视频。我们调用`VideoCapture`函数，根据视频文件路径打开视频。通过`cap.get`函数获得mp4视频的帧数。如下所示，该视频共有270帧。

In [3]:
cap = cv2.VideoCapture('boat.mp4')
NUM = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print("视频帧数： ",NUM)

视频帧数：  270.0


在进行检测前，需要设置相关参数。Lucas Kanade算法通过视频中角点的轨迹，进行光流跟踪。因此首先需要检测视频中的角点。我们采用Shi Tomasi算法进行角点检测。下面定义该算法的参数。

In [46]:
# 设置 Shi Tomasi角点检测的参数
st_params = dict(maxCorners=30, # 最大角点数
                qualityLevel=0.2, # 角点最低质量
                minDistance=2, # 角点间最小欧氏距离 
                blockSize=7)

#  设置 lucas kanade 光流场的参数
lk_params = dict(winSize=(15,15), # 搜索窗口大小
                maxLevel=2,       # 图像金字塔的层数
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1))

进行光流估计展示时，需要为检测出的角点定义颜色，以直观展示出角点轨迹的运动。这里我们选择了绿色。

In [47]:
# 定义角点的颜色
color = (0,255,0)  # RGB

使用`cap.read`函数，读取视频数据。第一个参数`ret`的返回值为`True`或`False`，表示是否读取到图像。第二个参数为`frame`，获得当前截取的一帧图像的数据。并将这一帧图像转换为灰度图后，调用`goodFeaturesToTrack`函数，检测图像中的角点。

In [48]:
# 获取第一帧图像数据
ret, first_frame = cap.read()

# 转换为灰度图
if ret == True:
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    
# 寻找图像中的角点
prev = cv2.goodFeaturesToTrack(prev_gray,mask = None,**st_params)

检测角点后，我们便可以对其进行光流跟踪。在此之前，为了图像展示的效果更好，需要对视频框的尺寸进行定义，避免图像过大。同时，绘制与视频内图像大小相同的掩膜，便于后续绘制角点的光流归记。

接着可以调用`calcOpticalFlowPyrLK`函数进行稀疏点光流估计。这里`while`循环，对视频每一帧进行处理，并打开窗口展示处理结果，直至视频播放完毕后，自动关闭窗口。

In [49]:
# 调整视频大小
cv2.namedWindow("Optical Flow",0)
cv2.resizeWindow("Optical Flow", 500, 500)

# 绘制和视频同样大小的掩膜
mask = np.zeros_like(first_frame) 

# 保存输出视频
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('稀疏点光流.mp4', fourcc, fps, size)

# frame_num控制while循环结束条件，视频播放完毕则关闭窗口
frame_num = 0
while(cap.isOpened):
    frame_num += 1
    if frame_num == NUM - 1:
        print("stop")  # 视频帧数读取完毕，关闭窗口
        break

    # 读取视频每帧图像
    ret, frame = cap.read()
    if ret == True:
        # 将图像转换为灰度图
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    else:
        break
   
    # 通过Lucas Kanade算法，进行光流跟踪，计算能够获取到的角点的新位置
    next, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **lk_params)
    
    # 选取好的角点，并筛选出旧的角点对应的新的角点
    # status=1表示找到特征点
    good_old = prev[status==1]
    good_new = next[status==1]
    
    # 绘制角点的轨迹
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel() # 返回新角点的坐标
        c,d = old.ravel() # 返回旧角点的坐标
        # 绘制线段连接新旧角点，展示角点的运动轨迹
        mask = cv2.line(mask, (a,b), (c,d), color, 2) 
        
        # 新旧角点用圆点表示，展示角点位置的变化
        frame = cv2.circle(frame, (a,b), 3, color, -1)
        
    # 将光流估计结果添加到原始图像上
    output = cv2.add(frame, mask)
    
    # 将旧图像（角点）更新为当前帧图像（角点），在下一次循环中，与下一帧图像相联系
    prev_gray = gray.copy()
    prev = good_new.reshape(-1,1,2)

    # 打开窗口展示光流跟踪结果
    cv2.imshow("Optical Flow", output)
    if cv2.waitKey(30) & 0xFF == ord("q"): # waitKey函数可以定义处理图像的速度
        break
    
    out.write(output)
    
# 关闭窗口
cap.release()
out.release()
cv2.destroyAllWindows()

<ipython-input-49-0ac2383c2b81>:43: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask, (a,b), (c,d), color, 2)
<ipython-input-49-0ac2383c2b81>:46: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame, (a,b), 3, color, -1)


stop


下面视频展示了稀疏点光流估计的结果。可以看到，图中绿色的原点表示每帧图像中显著点（角点）的位置，绿色的线段表示角点的运动轨迹。角点的运动轨迹与图中物体的运动轨迹相符合。

<div id="2.3"></div>
## 2.3 稠密光流

稠密光流计算图像上所有的点的偏移量，从而形成一个稠密的光流场。光流场，即物体在三维真实世界中的运动，在二维图像平面上的投影。通过稠密的光流场，可以进行像素级别的目标跟踪。稠密光流估计最经典的算法是Farneback算法，同样，我们介绍如何通过OpenCV实现稠密光流估计。

和稀疏点光流估计的步骤相似，我们首先读取视频数据，并读取第一帧图像，将其转换为灰度图。与稀疏点光流估计不同的是，此时不需要进行角点检测。

In [50]:
# 读取视频数据
cap = cv2.VideoCapture(r'boat.mp4')
NUM = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# 读取第一帧图像
ret , first_frame = cap.read()

# 转换为灰度图
if ret == True:
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# 定义掩膜，方便后续绘制运动轨迹
mask = np.zeros_like(first_frame)
mask[..., 1]= 255

类似地，我们通过`while`循环，逐帧分析视频，输出稠密光流估计的结果。

In [ ]:
# 设定两个窗口的尺寸，分别展示原始视频，及其对应的稠密光流估计视频
cv2.namedWindow("Dense Optical Flow",0)
cv2.resizeWindow("Dense Optical Flow", 500, 500)
cv2.namedWindow("input",0)
cv2.resizeWindow("input", 500, 500)

# 保存输出视频
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('稠密光流.mp4', fourcc, fps, size)

frame_num = 0 # frame_num控制while循环，视频结束时关闭窗口
while ( cap.isOpened()):
    frame_num += 1
    if frame_num == NUM - 1:
        print("stop!")  # 停止
        break
    
    # 读取图像
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('input', frame) # 打开窗口，展示原视频
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # 转换为灰度图像
    
    # 通过Farneback算法计算稠密光流估计，prev_gray为前一帧图像，gray为当前帧图像
    flow= cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5,3,15,3,5,1.2,0)
    
    # 计算梯度值和方向
    magn, angle = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    # 根据光流方向设置图像颜色
    mask[...,0] = angle*180/np.pi/2   
        
    # 将梯度值标准化
    mask[...,2] = cv2.normalize(magn, None, 0, 255, cv2.NORM_MINMAX)
    
    # 将HSV空间转换为RGB空间的图像
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2RGB)
    
    # 打开窗口，展示稠密光流估计结果
    cv2.imshow("Dense Optical Flow", rgb)
    
    # 将当前图像记为旧图像，为下一帧的计算做准备
    prev_gray = gray

    out.write(rgb)
    
    # cv2.waitKey能够控制图处理速度
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break 
    
# 关闭窗口
cap.release()
out.release()
cv2.destroyAllWindows()

下图为原视频及其稠密光流估计的结果。可以看到因为图像的梯度去掉了很多不必要的信息，加重了轮廓，通过稠密光流估计，可以很清晰地看到图中物体的轮廓，以及其运动轨迹。

<div id="2.4"></div>
## 2.4 算法实现

介绍了TLD算法的核心光流估计后，我们接下来看如何通过OpenCV，通过TLD算法，实现单目标跟踪。

调用`TrackerTLD_create`函数，使用TLD作为目标跟踪算法。

In [4]:
# 调用函数，选择TLD为目标跟踪算法
tracker = cv2.TrackerTLD_create()

# 读取跟踪器的名字，便于后续在视频中展示
tracker_name = str(tracker).split()[0][1:]

再读取视频，计算视频帧数，读取每帧图像数据。

In [5]:
cap = cv2.VideoCapture(r'boat.mp4')
NUM = cap.get(cv2.CAP_PROP_FRAME_COUNT) 
ret , frame= cap.read()

进行单目标跟踪，需要我们选择一个目标物体，算法才能继续计算该物体的运动轨迹。调用`selectROI`函数，可以打开之前读取的视频第一帧图像，并让使用者选择感兴趣的物体，后续进行目标跟踪。如下图，我们选择船只作为目标跟踪的对象。

之后，就可以用初始化目标跟踪算法的参数，并使用TLD算法分析视频，对刚才选择的物体进行目标跟踪。

In [ ]:
[ROI.png]

In [6]:
# 框出需要进行目标跟踪的物体
roi = cv2.selectROI(frame, False)

# 修改视频窗口大小
cv2.namedWindow(tracker_name,0)
cv2.resizeWindow(tracker_name, 500, 500)

# 读取目标数据，初始化目标跟踪参数
ret = tracker.init(frame, roi)

# 保存输出视频
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('TrackerTLD.mp4', fourcc, fps, size)

# 进行目标跟踪计算
frame_num = 0
while True:
    # 窗口关闭的条件：视频播放结束
    frame_num += 1
    if frame_num == NUM - 1:
        print("stop!") 
        break
        
    # 读取图像数据，跟踪的物体数据更新
    ret, frame = cap.read()
    success, roi = tracker.update(frame)
    (x,y,w,h) = tuple(map(int, roi))
    
    # 如果在当前帧图像中成功检测到了物体，框出物体位置
    if success: 
        pts1 = (x,y)
        pts2 = (x+w, y+h)
        cv2.rectangle(frame, pts1,pts2,(255,125,25),3)
    # 如果没有检测出物体，在图像中标注说明
    else: 
        cv2.putText(frame, '没有检测出物体', (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (25,125,255),3)
    
    # 在图像中说明使用的跟踪算法，此时为"TLD"算法
    cv2.putText(frame, tracker_name, (20,400),cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,0),3)
    
    # 打开窗口展示目标跟踪结果
    cv2.imshow(tracker_name, frame)    
    
    # 输出视频
    out.write(frame)
    
    # 退出循环
    if cv2.waitKey(30) & 0xFF == 27:
        break
        
# 关闭窗口
cap.release()
out.release()
cv2.destroyAllWindows()

stop!


下列视频展示了使用TLD算法进行目标跟踪的结果。在输出结果中，TLD算法检测并跟踪我们选择的目标船只，但是在后续船只被部分遮挡时，算法并没有很好地判断船只的位置和大小啊。接下来介绍更加主流，效果更为良好的目标跟踪算法：相关滤波算法。

<div id="3"></div>
# 3 MOSSE

相关滤波类方法是如今目标跟踪领域的主流算法。KCF、MOSSE都属于经典的相关滤波算法。

MOSSE开创性的将相关滤波技术引入到目标跟踪领域，可以很好的适应于复杂目标的旋转、被遮挡以及一些其他的干扰，且跟踪速度较快。

相关滤波源于信号处理领域，两个信号越相似，其相关值越高。在目标跟踪中，相关滤波算法的目的是：寻找滤波器，使得输入图像对滤波器的响应值尽可能大。

误差平方和最小的滤波器跟踪算法，首先确定第一帧中的目标为中心的一个小跟踪窗口，再同时训练跟踪器和滤波器。通过在下一帧图片的搜索窗口中，进行滤波来跟踪目标。滤波之后产生的最大值的地方就是目标的新位置。根据得到的新位置完成在线更新。

算法流程为[MOSSE.png]

我们定义函数，与实现TLD算法的思路一致，调用跟踪算法函数，并进行目标跟踪。

In [20]:
def tracking(tracker,input_path):
    # tracker: 跟踪算法； input_path：视频路径
    
    # 读取视频
    tracker_name = str(tracker).split()[0][1:]
    cap = cv2.VideoCapture(input_path)
    NUM = cap.get(cv2.CAP_PROP_FRAME_COUNT) 
    ret , frame= cap.read()
    
    # 框出需要进行目标跟踪的物体
    roi = cv2.selectROI(frame, False)
    
    # 修改视频窗口大小
    cv2.namedWindow(tracker_name,0)
    cv2.resizeWindow(tracker_name, 500, 500)
    
    ret = tracker.init(frame, roi)
    
    # 保存输出视频
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out = cv2.VideoWriter((tracker_name+".mp4"), fourcc, fps, size)
    
    # 进行目标跟踪计算
    frame_num = 0
    while True:
        # 窗口关闭的条件：视频播放结束
        frame_num += 1
        if frame_num == NUM - 1:
            print("stop!") 
            break
            
        # 读取图像数据，跟踪的物体数据更新
        ret, frame = cap.read()
        success, roi = tracker.update(frame)
        (x,y,w,h) = tuple(map(int, roi))
        
        # 如果在当前帧图像中成功检测到了物体，框出物体位置
        if success: 
            pts1 = (x,y)
            pts2 = (x+w, y+h)
            cv2.rectangle(frame, pts1,pts2,(255,125,25),3)
        else: 
            cv2.putText(frame, '没有检测出物体', (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (25,125,255),3)
        
        # 在图像中说明使用的跟踪算法
        cv2.putText(frame, tracker_name, (20,400),cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,0),3)
        cv2.imshow(tracker_name, frame)  # 打开窗口展示目标跟踪结果
        out.write(frame) # 写入视频文件
        if cv2.waitKey(30) & 0xFF == 27: # 退出循环
            break
    
    cap.release()
    cv2.destroyAllWindows() # 关闭窗口

同样选择船只作文跟踪目标，下列视频为MOSSE跟踪算法结果。观察到MOSSE跟踪效果良好，即使在视频后期，船只受到了其他物体的遮挡，MOSSE也准确地框出了目标。

In [31]:
# 调用函数，选择MOSSE为目标跟踪算法
tracker = cv2.TrackerMOSSE_create()
input_path = "boat.mp4"
tracking(tracker,input_path)

stop!


<div id="4"></div>
# 4 KCF

KCF算法在相关滤波基础上进行了优化，引入了循环矩阵。目标跟踪过程中缺少大量的负样本，通过循环矩阵可以增加负样本的数量，提高分类器训练的质量。

具体过程为：对目标区域进行循环移位，产生的图像作为分类器的训练样本，生成训练样本矩阵后，建立模型求解分类器。由此，将目标跟踪问题的求解，转化为分类问题 ，即对图像中的目标和背景分类。

下图为对原图像中的目标物体进行位移后，取得的新的样本图像。

In [ ]:
[KFC位移.png]

调用KCF目标跟踪算法，检测同一视频中船只的运动，观察跟踪效果。

In [42]:
# 调用函数，选择KCF为目标跟踪算法
tracker = cv2.TrackerKCF_create()
input_path = "boat.mp4"
tracking(tracker,input_path)

stop!


下列视频为使用KCF算法进行目标跟踪的效果，观察到蓝色矩形框中为我们选择的跟踪物体，KCF较为准确地计算出了船只的运动轨迹。

<div id="5"></div>
# 5 总结

本案例介绍了单目标跟踪算法的早期算法，即TLD算法。同时介绍了光流识别算法的内容，基于光流识别，TLD算法能够跟踪物体的运动。但这一算法的跟踪结果并不稳定，在物体遇到部分遮挡时效果较差。因此，之后介绍了如今更为主流的相关滤波算法MOSSE、KCF，其检测效果更加良好。在测试过程中，可以发现物体被遮挡地情况下，该算法也能够有效跟踪物体。